In [1]:
from pathlib import Path
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.agents.market_agent import MarketAgent
from market_agents.memory.agent_storage.agent_storage_api_utils import AgentStorageAPIUtils
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.agents.personas.persona import Persona
from market_agents.memory.document_reader import DocumentReaderFactory
from minference.lite.models import LLMConfig, ResponseFormat
from typing import List, Optional
from pydantic import BaseModel, Field

In [2]:
# Initialize storage and knowledge base
config_path = Path("../market_agents/memory/storage_config.yaml")
config = load_config_from_yaml(str(config_path))
config.stm_top_k = 3
config.ltm_top_k = 2
storage_utils = AgentStorageAPIUtils(config=config)

# Create knowledge base for SEC filings
sec_kb = MarketKnowledgeBase(
    config=config,
    table_prefix="sec_filings_kb"
)
await sec_kb.initialize()

# Create knowledge base agent
kb_agent = KnowledgeBaseAgent(
    market_kb=sec_kb,
    default_top_k=10)

In [3]:
config.model_dump()

{'storage_api_url': 'http://localhost:8001',
 'db_name': 'market_simulation',
 'user': 'db_user',
 'password': 'db_pwd@123',
 'host': 'localhost',
 'port': '5433',
 'pool_min': 1,
 'pool_max': 10,
 'index_method': 'ivfflat',
 'lists': 100,
 'embedding_api_url': 'https://api.openai.com/v1/embeddings',
 'model': 'text-embedding-ada-002',
 'batch_size': 32,
 'timeout': 10,
 'retry_attempts': 3,
 'retry_delay': 1.0,
 'retry_max_delay': 60,
 'retry_backoff_factor': 2.0,
 'retry_jitter': 0.1,
 'min_chunk_size': 512,
 'max_chunk_size': 1024,
 'vector_dim': 1536,
 'max_input': 4096,
 'top_k': 3,
 'similarity_threshold': 0.7,
 'encoding_format': 'float',
 'embedding_provider': 'openai',
 'stm_top_k': 3,
 'ltm_top_k': 2,
 'kb_top_k': 3}

In [4]:
file_path = Path("../examples/data/sec_filings/nvidia_10k.pdf")

reader = DocumentReaderFactory.get_reader(file_path)
text_content = reader.read(file_path)

# Store in knowledge base with metadata
knowledge_id = await kb_agent.store(
    text=text_content,
    metadata={
        "source": "SEC 10-K Filing",
        "type": "financial_filing",
        "file_path": str(file_path),
        "company": "NVIDIA Corporation"
    }
)

In [4]:
# Define environments
from market_agents.environments.mechanisms.research import ResearchEnvironment


class FinancialMetrics(BaseModel):
    """Key financial metrics from this reporting period"""

    period_end: str = Field(description="End date of reporting period")
    revenue: float = Field(description="Total revenue for period")
    revenue_growth: float = Field(description="Revenue growth percentage")
    gross_profit: Optional[float] = Field(None, description="Gross profit")
    gross_margin: float = Field(description="Gross margin percentage")
    operating_income: Optional[float] = Field(None, description="Operating income")
    operating_margin: Optional[float] = Field(
        None, description="Operating margin percentage"
    )
    net_income: float = Field(description="Net income")
    net_margin: Optional[float] = Field(None, description="Net margin percentage")
    eps: Optional[float] = Field(None, description="Basic earnings per share")
    diluted_eps: Optional[float] = Field(None, description="Diluted earnings per share")
    ebitda: Optional[float] = Field(
        None,
        description="EBITDA (Earnings Before Interest, Taxes, Depreciation, Amortization)",
    )
    free_cash_flow: Optional[float] = Field(None, description="Free cash flow")

research_env = ResearchEnvironment(
    name="sec_research",
    schema_model = FinancialMetrics,
    initial_topic="Extract finanical metrics for NVIDIA from 10-K filing",
    sub_rounds=1
)

In [6]:
# Create SEC Analyst Persona
sec_analyst_persona = Persona(
    name="David Miller",
    role="SEC Filing Analyst",
    persona="Detail-oriented financial analyst focused on extracting and structuring information from SEC filings",
    objectives=[
        "Extract key information from SEC filings",
        "Structure information into standardized format",
        "Identify important changes and trends",
        "Flag potential risks and material information"
    ],
    trader_type=["Expert", "Conservative", "Rational"],
    communication_style="Formal",
    routines=[
        "Review filing structure",
        "Extract key metrics and data",
        "Compare with historical filings",
        "Summarize key findings"
    ],
    skills=[
        "SEC filing analysis",
        "Financial statement analysis",
        "Data extraction",
        "Regulatory compliance"
    ]
)

# Create SEC Analyst Agent
sec_analyst = await MarketAgent.create(
    storage_utils=storage_utils,
    agent_id="sec_analyst",
    use_llm=True,
    llm_config=LLMConfig(
        model="gpt-4o",
        client="openai",
        response_format=ResponseFormat.tool,
        temperature=0.2,
        use_cache=True
    ),
    persona=sec_analyst_persona,
    knowledge_agent=kb_agent,
    environments={"sec_research": research_env} 
)

In [7]:
sec_analyst.model_dump()

/Users/interstellarninja/miniconda3/envs/market-agents/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `LocalObservation` but got `dict` with value `{}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'id': 'sec_analyst',
 'role': 'SEC Filing Analyst',
 'persona': 'Detail-oriented financial analyst focused on extracting and structuring information from SEC filings',
 'objectives': ['Extract key information from SEC filings',
  'Structure information into standardized format',
  'Identify important changes and trends',
  'Flag potential risks and material information'],
 'task': None,
 'tools': [],
 'llm_config': {'id': UUID('e7607587-5f0d-40fa-abba-9e667a8f33e5'),
  'created_at': datetime.datetime(2025, 3, 31, 14, 5, 36, 747792),
  'client': <LLMClient.openai: 'openai'>,
  'model': 'gpt-4o',
  'max_tokens': 400,
  'temperature': 0.2,
  'response_format': <ResponseFormat.tool: 'tool'>,
  'use_cache': True},
 'llm_orchestrator': <minference.lite.inference.InferenceOrchestrator at 0x117adb800>,
 'prompt_manager': <market_agents.agents.market_agent_prompter.MarketAgentPromptManager at 0x117a5e930>,
 'chat_thread': {'id': UUID('7feeb2f7-c13d-41de-a502-08223288ea9e'),
  'created_at': dat

In [8]:
results = await sec_analyst.run_episode(environment_name="sec_research")


Short-term Memory Results:
Memory 1:
{"monologue": "<think>In this environment, I am tasked with extracting financial metrics for NVIDIA from a 10-K filing. The environment is structured with a single round and step, indicating a straightforward task. The retrieved documents provide various sections of NVIDIA's 10-K filing, including certifications by executives, financial statements, and other regulatory disclosures. My objective is to identify and extract key financial metrics such as revenue, net income, and other relevant financial data from these documents. The presence of certifications and legal compliance sections suggests a focus on accuracy and regulatory adherence. I need to ensure that I capture the most relevant financial data while being mindful of any significant changes or risks highlighted in the filing. This task aligns with my persona as a detail-oriented financial analyst focused on SEC filings. I will proceed with a strategy to systematically review the documents,

INFO:root:Starting request #0



Retrieved Documents:
Document 1:
{'text': "Kress, Executive Vice President and Chief\nFinancial Officer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of her knowledge:\n.\n\nThe Company's Annual Report on Form 10-K for the year ended January 26, 2025, to which this Certification is attached as Exhibit 32.2 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n.\n\nThe information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.", 'similarity': 0.7634616471827455, 'context': "...a significant role in the registrant's internal\ncontrol over financial reporting.\n Date:  February 26, 2025\n \n/s/ COLETTE M. KRESS \nColette M. Kress\nExecutive Vice President and Chief Financial OfficerE

INFO:root:Parallel processing complete. Results saved to /Users/interstellarninja/Documents/projects/MarketAgents/outputs/inference_cache/openai_results_cc82ed22-f425-4e58-bbbc-eaf4211ef858_2025-03-31_10-06-04.jsonl
INFO:EntityRegistry:Parsing results from /Users/interstellarninja/Documents/projects/MarketAgents/outputs/inference_cache/openai_results_cc82ed22-f425-4e58-bbbc-eaf4211ef858_2025-03-31_10-06-04.jsonl
INFO:EntityRegistry:Converting result for chat_thread_id: 7feeb2f7-c13d-41de-a502-08223288ea9e
INFO:EntityRegistry:RawOutput(ead1b842-2056-4086-9ce7-e5b841272cdd): Registering entity
INFO:EntityRegistry:Successfully registered RawOutput(ead1b842-2056-4086-9ce7-e5b841272cdd)
INFO:EntityRegistry:RawOutput(ead1b842-2056-4086-9ce7-e5b841272cdd): Successfully registered
INFO:EntityRegistry:GeneratedJsonObject(e6ed710c-9a7f-415d-8a3e-69b95b3476d7): Registering entity
INFO:EntityRegistry:Successfully registered GeneratedJsonObject(e6ed710c-9a7f-415d-8a3e-69b95b3476d7)
INFO:EntityRegis

Total AI Requests: 1


...

ActionSpace type: <class 'market_agents.environments.mechanisms.research.ResearchActionSpace'>
ActionStep: Found 1 tools in action_space
ActionSpace has workflow attr: False
Chat thread before setup: format=ResponseFormat.tool, tools=['perception_tool']


INFO:root:Parallel processing complete. Results saved to /Users/interstellarninja/Documents/projects/MarketAgents/outputs/inference_cache/openai_results_f1923f70-476a-44e4-83d4-adcf65c90e50_2025-03-31_10-06-32.jsonl
INFO:EntityRegistry:Parsing results from /Users/interstellarninja/Documents/projects/MarketAgents/outputs/inference_cache/openai_results_f1923f70-476a-44e4-83d4-adcf65c90e50_2025-03-31_10-06-32.jsonl
INFO:EntityRegistry:Converting result for chat_thread_id: 7feeb2f7-c13d-41de-a502-08223288ea9e
INFO:EntityRegistry:RawOutput(38ceea3b-5820-4033-ba40-03c30e4e79de): Registering entity
INFO:EntityRegistry:Successfully registered RawOutput(38ceea3b-5820-4033-ba40-03c30e4e79de)
INFO:EntityRegistry:RawOutput(38ceea3b-5820-4033-ba40-03c30e4e79de): Successfully registered
INFO:EntityRegistry:GeneratedJsonObject(69059a3e-7494-4569-a85b-6edc5fe1e7e0): Registering entity
INFO:EntityRegistry:Successfully registered GeneratedJsonObject(69059a3e-7494-4569-a85b-6edc5fe1e7e0)
INFO:EntityRegis

Total AI Requests: 2


INFO:market_agents.agents.market_agent:Executing cognitive step: reflection
INFO:EntityRegistry:Starting parallel AI completion for 1 chat threads
INFO:EntityRegistry:Adding user message to ChatThread(7feeb2f7-c13d-41de-a502-08223288ea9e)
INFO:EntityRegistry:ChatMessage(4a659e04-f5a6-403c-be0a-ee130ec374df): Registering entity
INFO:EntityRegistry:Successfully registered ChatMessage(4a659e04-f5a6-403c-be0a-ee130ec374df)
INFO:EntityRegistry:ChatMessage(4a659e04-f5a6-403c-be0a-ee130ec374df): Successfully registered
INFO:EntityRegistry:Added user message(4a659e04-f5a6-403c-be0a-ee130ec374df) to ChatThread(7feeb2f7-c13d-41de-a502-08223288ea9e)
INFO:EntityRegistry:Preparing openai requests for 1 chat threads
INFO:EntityRegistry:Getting OpenAI request for ChatThread(7feeb2f7-c13d-41de-a502-08223288ea9e) with response format ResponseFormat.tool
INFO:EntityRegistry:Converting ChatThread(7feeb2f7-c13d-41de-a502-08223288ea9e) history to OpenAI format
INFO:EntityRegistry:Processing message: role=M

Total AI Requests: 3


In [9]:
import json


print(json.dumps(results, indent=2))

[
  {
    "monologue": "<think>The current environment involves extracting financial metrics from NVIDIA's 10-K filing. My previous attempt was partially successful, as I extracted key metrics like revenue, revenue growth, and net income, but missed others such as gross profit and operating income. The environment is structured with a single round and step, indicating a straightforward task. However, the incomplete extraction suggests I need to refine my approach to ensure all relevant financial metrics are captured. I will focus on improving my systematic review process and consider using additional resources if available. Confidence remains high, but I must be diligent in capturing all necessary data points to align fully with the task objectives.</think>",
    "key_observations": [
      "The task is to extract financial metrics for NVIDIA from a 10-K filing.",
      "The environment is structured with a single round and step, indicating a straightforward task.",
      "Previous ext